In [51]:
import json
f = open('../../../caueeg/dementia-no-overlap.json')
data_files = json.load(f)
f.close()
from IPython.display import clear_output


idx2label = {}
for file_idx in data_files["train_split"]:
    idx2label[file_idx["serial"]] = file_idx["class_label"]
for file_idx in data_files["test_split"]:
    idx2label[file_idx["serial"]] = file_idx["class_label"]
for file_idx in data_files["validation_split"]:
    idx2label[file_idx["serial"]] = file_idx["class_label"]
    
    
    
import glob, os
test_files = []
val_files = []
train_files = []
for file_name in data_files["test_split"]:
    test_files.append("../../../caueeg/signal/edf/"+file_name["serial"]+".edf")
for file_name in data_files["train_split"]:
    train_files.append("../../../caueeg/signal/edf/"+file_name["serial"]+".edf")
for file_name in data_files["validation_split"]:
    val_files.append("../../../caueeg/signal/edf/"+file_name["serial"]+".edf")
print(len(train_files))
print(len(val_files))
print(len(test_files))

#event_fname
import glob, os
all_events_files = []
for file in glob.glob("../../../caueeg/event/*.json"):
    all_events_files.append(file)
print(len(all_events_files))

events = {}
for file_name in all_events_files:
    file = open(file_name)
    event = json.load(file)
    file_name = "../../../caueeg/signal/edf/" + file_name.split("/")[-1][:-5]+".edf"
    events[file_name] = event
    file.close()

950
82
90
1379


In [53]:
import mne
from tqdm import tqdm
import random
import numpy as np
from scipy import signal
import spkit as sp
import spkit as sp

import scipy.signal as sig
from sklearn.decomposition import FastICA
    
from braindecode.preprocessing import exponential_moving_standardize


def get_label(file):
    idx = file.split("/")[-1][:-4]
    label = idx2label[idx]
    if label == 0:
        return [0]
    elif label == 1:
        return [1]
    else:
        return [2]
    
    
def read_edf_file(file, test=True):
    
    # load raw data
    raw = mne.io.read_raw(file, verbose=False, preload=True)
    raw.resample(sfreq=100)
    raw.filter(l_freq=1, h_freq=45, verbose=False)
    #raw.set_eeg_reference(ref_channels=['Photic'])
    
    # pick 19 channels
    ch_names = ['Fp1', 'F3', 'C3', 'P3', 'O1', 'Fp2', 'F4', 'C4', 'P4', 'O2', 'F7', 'T3', 'T5', 'F8', 'T4', 'T6', 'Fz', 'Pz', 'Cz']
    ch_types = ['eeg' for _ in range(len(ch_names))]
    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=100)
    
    raw.info = info
    picks_eeg = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False, exclude='bads')

    # set montage 10-20
    montage = mne.channels.make_standard_montage("standard_1020")
    raw.set_montage(montage)
    #raw.filter(1.0, 45.0, fir_design='firwin', n_jobs=1)
    #raw.resample(sfreq=100)
    #raw = mne.preprocessing.compute_current_source_density(raw)
    #raw.filter(1.0, 45.0, fir_design='firwin', n_jobs=1)
    raw = raw.get_data()

    #raw = (raw - raw.mean())/raw.std()
    
    #raw = raw.T
    #raw = sp.eeg.ICA_filtering(raw.copy(),verbose=0,winsize=100)
    #raw = raw.T
    #raw = raw.T
    #ica = FastICA(n_components=19, whiten="unit-variance")
    #raw = ica.fit_transform(raw)
    #raw = raw.T    
    #raw = exponential_moving_standardize(raw)
    raw = mne.filter.filter_data(raw, 100, 1.0, 45.0)
    dur = raw.shape[1]
    raws = []
    if test:
        raws.append(raw[:, 5000:10000])
    else:
        for i in range(3):
            if dur > ((i+1)*5000):
                raws.append(raw[:, i*5000:(i+1)*5000])
    return raws
    
    
def read_edf_file(file, use_windows=False, events="Eyes Closed"):
    
    # load raw data
    raw = mne.io.read_raw(file, verbose=False, preload=True)
    raw.filter(l_freq=1, h_freq=45, verbose=False)
    #raw.set_eeg_reference(ref_channels=['Photic'])

    # pick 19 channels
    ch_names = ['Fp1', 'F3', 'C3', 'P3', 'O1', 'Fp2', 'F4', 'C4', 'P4', 'O2', 'F7', 'T3', 'T5', 'F8', 'T4', 'T6', 'Fz', 'Pz', 'Cz']
    ch_types = ['eeg' for _ in range(len(ch_names))]
    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=200)
    raw.info = info
    picks_eeg = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False, exclude='bads')

    # set montage 10-20
    montage = mne.channels.make_standard_montage("standard_1020")
    raw.set_montage(montage)
    
    # filter
    #raw.filter(l_freq=0.5, h_freq=45, verbose=False)
    
    # events
    event_id = {}
    idx_ = 0
    for event in events[file]:
        if not event[1] in event_id.keys():
            event_id[event[1]] = idx_
            idx_ += 1 
            
    new_events = []
    label_events = []
    for event_idx, event in enumerate(events[file]):
        new_events.append(event[0])
        label_events.append(event_id[event[1]])  
        
    new_events = np.array(new_events)
    new_events = new_events.reshape(new_events.shape[0], 1).astype(int)
    zeros = np.array([0 for i in range(new_events.shape[0])]).reshape(new_events.shape[0], 1)
    label_events = np.array(label_events).reshape(new_events.shape[0], 1)
    new_events = np.concatenate((new_events, zeros, label_events), axis=1)

    # read epochs from raw file using events
    epochs = mne.Epochs(
        raw=raw,
        events=new_events,
        event_id=event_id,
        picks=picks_eeg,
        event_repeated="merge",
        preload=True,
        tmin=-0.01,
        detrend=0,
        tmax=30)
    
    epochs.filter(1.0, 45.0, fir_design='firwin', n_jobs=1)
    
    # csd of epochs file
    #epochs = mne.preprocessing.compute_current_source_density(epochs)
    epochs.resample(sfreq=100)

    # output epochs data
    if epochs.get_data().shape[0] < 5:
        return
    try:
        epochs = epochs[events]
        #print(epochs)
        epochs_data = epochs.get_data()
        epochs_data = epochs_data[:5]
        
    except:
        return
    
    if not use_windows:
        epochs_data = [epochs_data[int(len(epochs_data)/2)]]
        #epochs_data = np.array(epochs_data).squeeze()
        #epochs_data = epochs_data.T
        #epochs_data = sp.eeg.ICA_filtering(epochs_data.copy(),verbose=0,winsize=100)
        #epochs_data = epochs_data.T
        return epochs_data
    else:
        return epochs_data
    

def build_data(raw_data, use_windows=True):
    
    all_data_features = []
    data_labels = []
    
    for file in tqdm(raw_data):
        
        #node features
        sample_features = []
        tfeatures = []
        
        edf_data = read_edf_file(file, use_windows)
        if edf_data is None: 
            continue
            
        for edf_data1 in edf_data:
            all_data_features.append(edf_data1)
            label = get_label(file)
            data_labels.append(np.array(label))

    all_data_features = np.array(all_data_features)
    data_labels = np.array(data_labels)
             
    return all_data_features, data_labels

In [16]:
import copy

from sklearn.preprocessing import OneHotEncoder
#from train import trainer
import torch
#from tgcn import TGCN
from torch.utils.data import TensorDataset,DataLoader
from scipy.stats import pearsonr

from numpy.lib.stride_tricks import sliding_window_view
from scipy import signal
from statsmodels.tsa.stattools import grangercausalitytests
from tqdm import tqdm


def multichannel_sliding_window(X, size, step):
    shape = (X.shape[0] - X.shape[0] + 1, (X.shape[1] - size + 1) // step, X.shape[0], size)
    strides = (X.strides[0], X.strides[1] * step, X.strides[0], X.strides[1])
    return np.lib.stride_tricks.as_strided(X, shape, strides)[0]


def hilphase(x1,x2):
    sig1_hill=sig.hilbert(x1)
    sig2_hill=sig.hilbert(x2)
    pdt=(np.inner(sig1_hill,np.conj(sig2_hill))/(np.sqrt(np.inner(sig1_hill,
               np.conj(sig1_hill))*np.inner(sig2_hill,np.conj(sig2_hill)))))
    phase = np.angle(pdt)
    return phase
    
def gc(x1, x2):
    X = np.vstack([x1, x2]).T
    gc = grangercausalitytests(X, [2], addconst=True, verbose=False)[2][0]['ssr_ftest'][1]
    return gc
    
    
# Coherence - δ
def coherence(eegData,fs):
    coh_res = []
    for ii, jj in itertools.combinations(range(eegData.shape[0]), 2):
        coh_res.append(CoherenceDelta(eegData, ii, jj, fs=fs))
    coh_res = np.array(coh_res)
    return coh_res

# Mutual information
from sklearn.feature_selection import mutual_info_regression

def cal_mi(x1, x2):
    mi = mutual_info_regression(x1, x2)
    return mi

# Mutual information
def calculate2Chan_MI(eegData,ii,jj,bin_min=-200, bin_max=200, binWidth=2):
    H = np.zeros(eegData.shape[2])
    bins = np.arange(bin_min+1, bin_max, binWidth)
    for epoch in range(eegData.shape[2]):
        c_xy = np.histogram2d(eegData[ii,:,epoch],eegData[jj,:,epoch],bins)[0]
        H[epoch] = mutual_info_score(None, None, contingency=c_xy)
    return H


def gen_graphs(eegs, num_nodes=19, cal_conn="corr"):
    #eegs(snapshots, bands, timpoints)
    c = []
    for i in range(num_nodes):
        c1 = []
        for j in range(num_nodes):
            if cal_conn == "corr":
                conn = pearsonr(eegs[i], eegs[j])[0]
                #conn = np.correlate(eegs[i], eegs[j], mode='valid')[0]
            elif cal_conn == "plv": 
                conn = hilphase(eegs[i], eegs[j])
            elif cal_conn == "gc": 
                conn = gc(eegs[i], eegs[j])
            elif cal_conn == "mi": 
                eegsi = eegs[i].reshape(eegs[i].shape[0], 1)
                eegsj = eegs[j].reshape(eegs[j].shape[0], 1)
                conn = cal_mi(eegsi, eegsj)
            elif cal_conn == "coh": 
                conn = coherence(eegs[i], eegs[j])
            c1.append(conn)
        c.append(c1)
    return c


def gen_features(X, y, device, cal_conn, window_size=100, overlap=50, augment=False):
    X_new = []     
    #freqs = np.where((f >= 1) & (f <= 30))

    for x in X:
        x_ = multichannel_sliding_window(x, window_size, overlap)
        X_new.append(x_)
        
    X_new = np.moveaxis(np.array(X_new), 1, -1)
    
    graphs = []
    dynamic_range = X_new.shape[-1]
    
    threshold = 0.7

    print("calculating connectivity")
    for x in tqdm(X_new):
        temp = []
        for i in range(dynamic_range):
            temp_g = gen_graphs(x[:, :, i], cal_conn=cal_conn)
            temp_g = np.array(temp_g)
            temp_g = (temp_g - temp_g.min())/(temp_g.max() - temp_g.min())
            #temp_g[temp_g<threshold] = 0
            temp.append(temp_g)
        graphs.append(temp)
        
    graphs = np.array(graphs)
    
    graphs = np.moveaxis(graphs.squeeze(), 1, -1)
            
    return X_new, graphs, y

import copy
def standardize_data(train_X, test_X, val_X):
    train_X_std = copy.deepcopy(train_X)
    test_X_std = copy.deepcopy(test_X)
    val_X_std = copy.deepcopy(val_X)
    for i in range(train_X.shape[1]):
        min_ = np.min(train_X[:, i, :])
        max_ = np.max(train_X[:, i, :])
        train_X_std[:, i, :] = (train_X[:, i, :] - min_)/(max_ - min_)
        test_X_std[:, i, :] = (test_X[:, i, :] - min_)/(max_ - min_)
        val_X_std[:, i, :] = (val_X[:, i, :] - min_)/(max_ - min_)
    return train_X_std, test_X_std, val_X_std

In [28]:
from IPython.display import clear_output
from torch.utils.data import TensorDataset,DataLoader
from sklearn.preprocessing import OneHotEncoder


def train_test(train_files, test_files, val_files, cal_conn):
    print("read_data")
    train_X, train_y = build_data(train_files, use_windows=True)
    test_X, test_y = build_data(test_files, use_windows=False)
    val_X, val_y = build_data(val_files, use_windows=False)
    
    clear_output()
    print(train_X.shape, test_X.shape, val_X.shape)
    print("standardize_data")
    train_X, test_X, val_X = standardize_data(train_X, test_X, val_X)
    
    DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    train_X, train_graphs, train_y = gen_features(train_X, train_y, DEVICE, cal_conn=cal_conn, 
                                                  window_size=100, overlap=50, augment=False)
    test_X, test_graphs, test_y = gen_features(test_X, test_y, DEVICE, cal_conn=cal_conn, 
                                               window_size=100, overlap=50, augment=False)
    val_X, val_graphs, val_y = gen_features(val_X, val_y, DEVICE, cal_conn=cal_conn, 
                                               window_size=100, overlap=50, augment=False)
    
    clear_output()
    encoder = OneHotEncoder()
    train_y = encoder.fit_transform(train_y).toarray()
    test_y = encoder.transform(test_y).toarray()
    val_y = encoder.transform(val_y).toarray()
    train_X = torch.Tensor(train_X)
    test_X = torch.Tensor(test_X)
    val_X = torch.Tensor(val_X)
    train_y= torch.Tensor(train_y)
    test_y = torch.Tensor(test_y)
    val_y = torch.Tensor(val_y)
    test_graphs= torch.Tensor(test_graphs)
    train_graphs = torch.Tensor(train_graphs)
    val_graphs = torch.Tensor(val_graphs)
    
    batch_size = 64
    data = TensorDataset(train_X, train_graphs, train_y)
    train_iter = torch.utils.data.DataLoader(data, batch_size, shuffle=True)
    data = TensorDataset(test_X, test_graphs, test_y)
    test_iter = torch.utils.data.DataLoader(data, batch_size, shuffle=False)
    data = TensorDataset(val_X, val_graphs, val_y)
    val_iter = torch.utils.data.DataLoader(data, batch_size, shuffle=False)
    return train_iter, test_iter, val_iter

In [29]:
train_iter, test_iter, val_iter = train_test(train_files=train_files, 
                                   test_files=test_files, 
                                   val_files=val_files, 
                                   cal_conn="corr")


In [65]:
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool
import torch.nn as nn
import torch
from torch_scatter import scatter_add
from torch_geometric.utils import add_self_loops

def get_laplacian(edge_index, edge_weight=None, normalization=None, dtype=None,
                  num_nodes=19):
    
    assert normalization in [None, 'sym', 'rw'], 'Invalid normalization'

    if edge_weight is None:
        edge_weight = torch.ones(edge_index.size(1), dtype=dtype,
                                 device=edge_index.device)

    row, col = edge_index
    deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)

    if normalization is None:
        # L = D - A.
        edge_index, _ = add_self_loops(edge_index, num_nodes=num_nodes)
        edge_weight = torch.cat([-edge_weight, deg], dim=0)
    elif normalization == 'sym':
        # Compute A_norm = -D^{-1/2} A D^{-1/2}.
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        edge_weight = deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

        # L = I - A_norm.
        edge_index, edge_weight = add_self_loops(edge_index, -edge_weight,
                                                 fill_value=1,
                                                 num_nodes=num_nodes)
    else:
        # Compute A_norm = -D^{-1} A.
        deg_inv = 1.0 / deg
        deg_inv[deg_inv == float('inf')] = 0
        edge_weight = deg_inv[row] * edge_weight

        # L = I - A_norm.
        edge_index, edge_weight = add_self_loops(edge_index, -edge_weight,
                                                 fill_value=1,
                                                 num_nodes=num_nodes)

    return edge_index, edge_weight

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, auc, roc_auc_score
import torch.nn.functional as F
import torch.nn as nn

import torch
from torch_geometric.nn import GCNConv

import torch
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool
import torch.nn as nn


class TGCN2(torch.nn.Module):
    def __init__(self, in_channels: int, out_channels: int, 
                 improved: bool = False, cached: bool = False, 
                 add_self_loops: bool = True):
        super(TGCN2, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved
        self.cached = cached
        self.add_self_loops = add_self_loops
        self._create_parameters_and_layers()

    def _create_update_gate_parameters_and_layers(self):
        self.conv_z = GCNConv(in_channels=self.in_channels,  out_channels=self.out_channels, improved=self.improved,
                              cached=self.cached, add_self_loops=self.add_self_loops )
        self.linear_z = torch.nn.Linear(2 * self.out_channels, self.out_channels)

    def _create_reset_gate_parameters_and_layers(self):
        self.conv_r = GCNConv(in_channels=self.in_channels, out_channels=self.out_channels, improved=self.improved,
                              cached=self.cached, add_self_loops=self.add_self_loops )
        self.linear_r = torch.nn.Linear(2 * self.out_channels, self.out_channels)

    def _create_candidate_state_parameters_and_layers(self):
        self.conv_h = GCNConv(in_channels=self.in_channels, out_channels=self.out_channels, improved=self.improved,
                              cached=self.cached, add_self_loops=self.add_self_loops )
        self.linear_h = torch.nn.Linear(2 * self.out_channels, self.out_channels)

    def _create_parameters_and_layers(self):
        self._create_update_gate_parameters_and_layers()
        self._create_reset_gate_parameters_and_layers()
        self._create_candidate_state_parameters_and_layers()

    def _set_hidden_state(self, X, H):
        if H is None:
            # can infer batch_size from X.shape, because X is [B, N, F]
            H = torch.zeros(X.shape[0], X.shape[1], self.out_channels).to(X.device) #(b, 207, 32)
        return H

    def _calculate_update_gate(self, X, edge_index, edge_weight, H):
        Z = torch.cat([self.conv_z(X, edge_index, edge_weight), H], axis=2) # (b, 207, 64)
        Z = self.linear_z(Z) # (b, 207, 32)
        Z = torch.sigmoid(Z)

        return Z

    def _calculate_reset_gate(self, X, edge_index, edge_weight, H):
        R = torch.cat([self.conv_r(X, edge_index, edge_weight), H], axis=2) # (b, 207, 64)
        R = self.linear_r(R) # (b, 207, 32)
        R = torch.sigmoid(R)

        return R

    def _calculate_candidate_state(self, X, edge_index, edge_weight, H, R):
        H_tilde = torch.cat([self.conv_h(X, edge_index, edge_weight), H * R], axis=2) # (b, 207, 64)
        H_tilde = self.linear_h(H_tilde) # (b, 207, 32)
        H_tilde = torch.tanh(H_tilde)

        return H_tilde

    def _calculate_hidden_state(self, Z, H, H_tilde):
        H = Z * H + (1 - Z) * H_tilde   # # (b, 207, 32)
        return H

    def forward(self,X: torch.FloatTensor, edge_index: torch.LongTensor, edge_weight: torch.FloatTensor = None,
                H: torch.FloatTensor = None ) -> torch.FloatTensor:
       
        H = self._set_hidden_state(X, H)
        Z = self._calculate_update_gate(X, edge_index, edge_weight, H)
        R = self._calculate_reset_gate(X, edge_index, edge_weight, H)
        H_tilde = self._calculate_candidate_state(X, edge_index, edge_weight, H, R)
        H = self._calculate_hidden_state(Z, H, H_tilde) # (b, 207, 32)
        return H
    


class A3TGCN2(torch.nn.Module):
    def __init__(
        self,
        in_channels: int, 
        out_channels: int,  
        periods: int, 
        improved: bool = False,
        cached: bool = False,
        add_self_loops: bool = True):
        super(A3TGCN2, self).__init__()

        self.in_channels = in_channels  # 2
        self.out_channels = out_channels # 32
        self.periods = periods # 12
        self.improved = improved
        self.cached = cached
        self.add_self_loops = add_self_loops
        self._setup_layers()

    def _setup_layers(self):
        self._base_tgcn = TGCN2(
            in_channels=3,
            out_channels=self.out_channels,  
            improved=self.improved,
            cached=self.cached, 
            add_self_loops=self.add_self_loops)

        self.Encoder =  Encoder(timepoints=100, num_nodes=19)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self._attention = torch.nn.Parameter(torch.empty(self.periods, device=device))
        torch.nn.init.uniform_(self._attention)

    def forward( 
        self, 
        X: torch.FloatTensor,
        A: torch.FloatTensor, 
        edge_weight: torch.FloatTensor = None,
        H: torch.FloatTensor = None
    ) -> torch.FloatTensor:

        
        Xenc = self.Encoder(X)
        H_accum = 0
        probs = torch.nn.functional.softmax(self._attention, dim=0)

        for period in range(self.periods):
            Xt = Xenc[:, :, :, period]
            At = A[:, :, :, period]
            A_diag = torch.block_diag(*At)
            shape = A_diag.shape[0]
            idx = torch.ones(2, shape*shape).long().to(X.device)
            w = A_diag.flatten().float().to(X.device)
            idx, w = get_laplacian(idx, w, normalization="rw")
            H_accum = H_accum + probs[period] * self._base_tgcn(Xt, idx, w, H) #([32, 207, 32]
        return H_accum
        
class EEGModel(nn.Module):
    def __init__(self, num_nodes, node_features, num_classes, num_windows, device):
        super(EEGModel, self).__init__()
        self.a3tgnn = A3TGCN2(in_channels=node_features, out_channels=32, periods=num_windows) # node_features=2, periods=12
        self.num_nodes = num_nodes
        self.BN = nn.BatchNorm1d(self.num_nodes)
        self.num_windows = num_windows
        self.fc2 = nn.Linear(self.num_nodes*32, num_classes)
        self.softmax = nn.LogSoftmax(-1)
        
    def forward(self, X, A):
        HS = self.a3tgnn(X, A)
        HS = nn.functional.relu(HS)
        HS = self.BN(HS)
        HS = HS.reshape(HS.shape[0], self.num_nodes*32)
        out = self.fc2(HS)
        out = self.softmax(out)
        return out



class Encoder(nn.Module):
    def __init__(self, timepoints, num_nodes):
        super().__init__()
        self.conv1 = nn.Conv2d(num_nodes, 64, kernel_size=(3, 1), stride=(2, 1))
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 1))
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(3, 1), stride=(2, 1))
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 1))
        self.fc1 = nn.Linear(128, num_nodes)
        self.fc2 = nn.Linear(5, 5)
        #self.pooling = torch.nn.AdaptiveAvgPool2d((19, 3))
        
    def forward(self,x):
        x = nn.functional.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = nn.functional.relu(self.conv3(x))
        x = self.maxpool2(x)
        x = x.permute(0, 3, 2, 1)
        x = self.fc1(x)
        x = x.permute(0, 1, 3, 2)
        x = self.fc2(x)
        x = x.permute(0, 2, 3, 1)
        return x


class Encoder(nn.Module):
    def __init__(self, timepoints, num_nodes):
        super().__init__()
        self.conv1 = nn.Conv2d(num_nodes, 32, kernel_size=(3, 1), stride=(2, 1))
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 1))
        self.conv3 = nn.Conv2d(32, 64, kernel_size=(3, 1), stride=(2, 1))
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 1))
        self.pooling = torch.nn.AdaptiveAvgPool2d((num_nodes, 3))
        self.BN1 = nn.BatchNorm2d(64)
        self.BN2 = nn.BatchNorm2d(num_nodes)
        
    def forward(self,x):
        x = nn.functional.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = nn.functional.relu(self.conv3(x))
        x = self.maxpool2(x)
        x = self.BN1(x)
        x = x.permute(0, 3, 1, 2)
        x = self.pooling(x)
        x = x.permute(0, 2, 3, 1)
        x = self.BN2(x)
        return x

In [ ]:
def train_model(model, train_iter, val_iter, num_epochs, lr):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    opt_loss = np.inf
    print("Training model")
    model.train()
    for epoch in range(num_epochs):
        losses = 0
        for idx, (X, A, y) in enumerate(tqdm(train_iter)):
            optimizer.zero_grad()
            out = model(X, A)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            losses += loss.item()*X.shape[0]
        print("Epoch ", epoch+1, ":")
        print("train loss:", losses/len(train_iter.dataset))

        losses = 0
        for idx, (X, A, y) in enumerate(val_iter):
            out = model(X, A)
            loss = criterion(out, y)
            losses += loss.item()*X.shape[0]
        print("val loss:", losses/len(val_iter.dataset))

        if losses/len(val_iter.dataset) < opt_loss:
            torch.save(model.state_dict(), "saved_model/model.pt")
            opt_loss = losses/len(val_iter.dataset) 

model = EEGModel(num_nodes=19, node_features=100, num_classes=3, num_windows=58, device=DEVICE)
train_model(model, train_iter, val_iter, num_epochs=20, lr=1e-2)

Training model


 92%|███████████████████████████████████████▍   | 66/72 [01:37<00:09,  1.51s/it]

In [68]:
def eval_model(model_path, data_iter):
    outs= []
    ys = []

    model = EEGModel(num_nodes=19, node_features=100, num_classes=3, num_windows=58, device=DEVICE)
    model.load_state_dict(torch.load(model_path, weights_only=False))   

    for X, A, y in data_iter:
        out = model(X, A)
        outs.extend(out.cpu().detach().numpy())
        ys.extend(y.cpu().detach().numpy())

    outs = np.array(outs)
    ys = np.array(ys)

    print(accuracy_score(np.argmax(outs, -1), np.argmax(ys, -1)), 
          confusion_matrix(np.argmax(outs, -1), np.argmax(ys, -1)))
    
eval_model("saved_model/model.pt", test_iter)

0.5955056179775281 [[16  6  3]
 [16 22  4]
 [ 2  5 15]]
